# 01 Data preparation

Processing xlsx files from data folder into suitable inputs and generate other input files

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [29]:
cell_line ='CAL29'
data_dir = f"/home/jing/Phd_project/project_UCD_blca/blca_publication_OUTPUT/blca_publication_OUTPUT_LINCS/00_outputs_2020_{cell_line}/"
info_dir = data_dir
out_dir = f"/home/jing/Phd_project/project_UCD_blca/blca_publication_OUTPUT/blca_publication_OUTPUT_bmra/blca_publication_OUTPUT_bmra_{cell_line}/00_outputs_2020_{cell_line}"

#os.makedirs(info_dir, exist_ok = True)
os.makedirs(out_dir, exist_ok = True)

## Modules

Load data about modules and drugs.

In [4]:
### DATA
### remove whitespaces in names (modules), remove duplicates

modules_df = pd.read_excel(
    os.path.join(info_dir, "ALL_DATA_2020_Jing.xlsx"), sheet_name = "modules", index_col = 0)
# display(modules_df)

selected_modules = modules_df.index.tolist()
print(len(selected_modules), ' - Size after reading')

# remove duplicates
selected_modules = modules_df.index.unique().tolist()
print(len(selected_modules), ' - Size after remove duplicates')

# remove whitespaces in modules' names 
selected_modules = [d.strip() for d in selected_modules]

print('Selected modules list: ', len(selected_modules), selected_modules)


10  - Size after reading
10  - Size after remove duplicates
Selected modules list:  10 ['CDK1', 'CDK2', 'CDK4_6', 'EGFR', 'PI3K', 'FGFR', 'TOP2A', 'p53', 'Estrogen', 'Androgen']


In [6]:
### DATA
### remove whitespaces in names (modules, drugs), remove duplicates
### check the dimensions of the indicator IC50 1 uM = 1000 nM
### copy-paste as values, numbers, no formulas

IC50_df = pd.read_excel(
    os.path.join(info_dir, "ALL_DATA_2020_Jing.xlsx"), sheet_name = "IC50s")
IC50_df.drop(columns=['Unnamed: 3','Unnamed: 4'],inplace=True)

print(len(IC50_df.index), ' - Size after reading')
# display(IC50_df)

# rename
IC50_df = IC50_df.rename(columns = {"IC50, uM": "IC50"})

# manually correcting value IC50, Example  for IOX2 -> 30 nM  
#IC50_df.loc[IC50_df.index == 'IOX2', IC50_df.columns == 'IC50'] = 30/1000

# remove non-selected modules, modules' names with whitespaces or empty
IC50_df = IC50_df[IC50_df.Module.isin(selected_modules)]

print(len(IC50_df.index), ' - Size after remove modules')
# display(IC50_df)

# remove duplicates 
# Considering certain columns is optional. 
# Indexes, including time indexes are ignored.
IC50_df = IC50_df.drop_duplicates()

print(len(IC50_df.index), ' - Size after remove duplicates')
display(IC50_df)

39  - Size after reading
39  - Size after remove modules
39  - Size after remove duplicates


,Drug,Module,IC50
0,flufenamic-acid,Androgen,3.00000
1,nandrolone,Androgen,9.00000
2,oxandrolone,Androgen,190.30000
3,testosterone-enanthate,Androgen,200.00000
4,testosterone-propionate,Androgen,124.00000
5,JNJ-7706621,CDK1,0.02700
6,PHA-793887,CDK1,0.18000
7,roscovitine,CDK2,2.00000
8,alvocidib,CDK4_6,0.12000
9,palbociclib,CDK4_6,0.04500


In [7]:
modules = IC50_df.Module.unique().tolist()

print('IC50_df  modules list: ', len(modules), modules)
print()
print('Selected modules list: ', len(selected_modules), selected_modules)

### CHECK
print()
print('CHECK: ', len(selected_modules),'=?', len(modules))

n_modules = len(modules)


IC50_df  modules list:  10 ['Androgen', 'CDK1', 'CDK2', 'CDK4_6', 'EGFR', 'Estrogen', 'FGFR', 'PI3K', 'p53', 'TOP2A']

Selected modules list:  10 ['CDK1', 'CDK2', 'CDK4_6', 'EGFR', 'PI3K', 'FGFR', 'TOP2A', 'p53', 'Estrogen', 'Androgen']

CHECK:  10 =? 10


In [8]:
drugs = IC50_df.Drug.tolist()
print(len(drugs), ' - Size after reading')

# remove duplicates
drugs = IC50_df.Drug.unique().tolist()
print(len(drugs), ' - Size after remove duplicates')

# remove whitespaces in drugs' names (necessary for some)
drugs = [d.strip() for d in drugs]

# remove duplicates after remove whitespaces
drugs = list(set(drugs))
print(len(drugs), ' - Size after remove duplicates without whitespaces')

print('Drugs list: ', len(drugs), drugs)

n_drugs = len(drugs)

39  - Size after reading
39  - Size after remove duplicates
39  - Size after remove duplicates without whitespaces
Drugs list:  39 ['serdemetan', 'testosterone-propionate', 'NVP-BEZ235', 'estradiol-cypionate', 'PI-103', 'LY-294002', 'roscovitine', 'mitoxantrone', 'alvocidib', 'RITA', 'erlotinib', 'masitinib', 'HLI-373', 'testosterone-enanthate', 'afatinib', 'GDC-0349', 'nandrolone', 'ponatinib', 'idarubicin', 'sorafenib', 'epirubicin', 'taselisib', 'daunorubicin', 'AS-605240', 'oxandrolone', 'dienestrol', 'vandetanib', 'raloxifene', 'gefitinib', 'flufenamic-acid', 'SAR405838', 'KU-0063794', 'lapatinib', 'PHA-793887', 'palbociclib', 'AZD-8055', 'nutlin-3', 'JNJ-7706621', 'AMG-232']


## L1000 meta data

Get sig_id for selected drugs.

In [9]:
sig_info_df = pd.read_excel(os.path.join(data_dir, f"sig_info_2020_{cell_line}.xlsx"), index_col = 0)

display(sig_info_df)

,cell,plate,time,level_3_samples,samples_number,pert_type,pert_drug,targets,targets_number,dose,dose_float
level_5_sig_id,,,,,,,,,,,
ASG002_CAL29_24H:A03,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A03,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
ASG002_CAL29_24H:A04,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A04,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
ASG002_CAL29_24H:A05,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A05,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
ASG002_CAL29_24H:A06,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A06,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
ASG002_CAL29_24H:J13,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:J13,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
...,...,...,...,...,...,...,...,...,...,...,...
MOAR012_CAL29_24H:P20,CAL29,MOAR012,24 h,MOAR012_CAL29_24H_X1_B35:P20,1,trt_cp,BAY-61-3606,NaN,0,3.33 uM,3.33
MOAR012_CAL29_24H:P21,CAL29,MOAR012,24 h,MOAR012_CAL29_24H_X1_B35:P21,1,trt_cp,BAY-61-3606,NaN,0,1.11 uM,1.11
MOAR012_CAL29_24H:P22,CAL29,MOAR012,24 h,MOAR012_CAL29_24H_X1_B35:P22,1,trt_cp,ethaverine,NaN,0,10 uM,10.00


In [10]:
# now filtering so only the required drugs are present
sig_info_df = sig_info_df.loc[sig_info_df.pert_drug.isin(drugs)]

# here's what we have now
display(sig_info_df)

,cell,plate,time,level_3_samples,samples_number,pert_type,pert_drug,targets,targets_number,dose,dose_float
level_5_sig_id,,,,,,,,,,,
ASG002_CAL29_24H:A10,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A10,1,trt_cp,taselisib,PIK3CA,1,10 uM,10.00
ASG002_CAL29_24H:A11,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A11,1,trt_cp,taselisib,PIK3CA,1,1.11 uM,1.11
ASG002_CAL29_24H:A12,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A12,1,trt_cp,taselisib,PIK3CA,1,0.12 uM,0.12
ASG002_CAL29_24H:A19,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A19,1,trt_cp,AS-605240,PIK3CG,1,10 uM,10.00
ASG002_CAL29_24H:A20,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A20,1,trt_cp,AS-605240,PIK3CG,1,1.11 uM,1.11
...,...,...,...,...,...,...,...,...,...,...,...
MOAR011_CAL29_24H:F08,CAL29,MOAR011,24 h,MOAR011_CAL29_24H_X1_B35:F08,1,trt_cp,serdemetan,MDM2,1,3.33 uM,3.33
MOAR011_CAL29_24H:F09,CAL29,MOAR011,24 h,MOAR011_CAL29_24H_X1_B35:F09,1,trt_cp,serdemetan,MDM2,1,1.11 uM,1.11
MOAR011_CAL29_24H:J10,CAL29,MOAR011,24 h,MOAR011_CAL29_24H_X1_B35:J10,1,trt_cp,nandrolone,"AR, CYP19A1, MAOA, MAOB, NISCH",5,10 uM,10.00


Manually remove  few inhibition from data set, since it does differ from the other data points.

inhib_to_filter = "PF-03758309"
dose_to_filter = 10

id_to_filter = sig_info_df[
    np.logical_and(
        sig_info_df.pert_drug == inhib_to_filter,
        sig_info_df.dose_float == dose_to_filter,
    )
].index.values

print(id_to_filter)

sig_info_df = sig_info_df[~sig_info_df.index.isin(id_to_filter)]
display(sig_info_df)

inhib_to_filter = "roscovitine"
dose_to_filter = 3.33

id_to_filter = sig_info_df[
    np.logical_and(
        sig_info_df.pert_drug == inhib_to_filter,
        sig_info_df.dose_float == dose_to_filter,
    )
].index.values

print(id_to_filter)

sig_info_df = sig_info_df[~sig_info_df.index.isin(id_to_filter)]
display(sig_info_df)

In [11]:
exp_ids = sig_info_df.index.unique().tolist()
print('Experiments ids list: ', len(exp_ids))

n_experiments = len(exp_ids)

Experiments ids list:  116


Confirm data by checking the drugs of interest against the filtered L1000 meta data.

In [12]:
print(f"Number of drugs of interest:\t{len(drugs)}")
#print(f'Number of drugs in L1000 data:\t{len(sig_info_df.value_counts("drugs"))}')

#sig_info_df.value_counts("drugs")

Number of drugs of interest:	39


## L1000 data

In [13]:
Data_norm_df = pd.read_excel(os.path.join(data_dir, f"Data_norm_2020_{cell_line}.xlsx"), index_col = 0)
display(Data_norm_df)

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
ASG002_CAL29_24H:A03,-0.080008,0.299000,-0.358535,0.607407,0.273957,-0.342600,0.485377,0.194704,0.127850,-0.124635,...,0.796081,0.527034,-0.923981,-0.914088,-0.130942,-0.646923,-0.512850,0.228304,0.214569,0.365473
ASG002_CAL29_24H:A04,-0.328507,0.145749,-0.088684,-0.133493,-0.053592,0.022400,0.081677,-0.389596,-0.042950,0.124016,...,0.561081,0.181935,0.395420,-0.736189,0.043908,0.600277,0.065350,0.207504,-0.078031,0.359373
ASG002_CAL29_24H:A05,-0.094208,0.111500,1.150316,0.198558,0.155808,-0.164900,0.076427,0.271704,0.193250,0.199615,...,0.141381,-0.094615,0.395119,-1.137388,0.046408,-0.970223,-0.218301,0.343904,-0.209031,-0.378327
ASG002_CAL29_24H:A06,-0.328857,0.200950,1.132116,-0.001092,0.050158,-0.239000,-0.084523,-0.675446,-0.188350,-0.258985,...,-0.725219,-0.334565,0.515119,-1.421788,-0.088892,0.345577,0.091350,-0.014446,0.251569,0.299674
ASG002_CAL29_24H:J13,0.384643,-0.083000,0.674916,-0.336792,0.453207,-0.029100,-0.169523,-0.452997,0.201750,0.177415,...,-0.578620,0.041284,-0.987181,0.581712,-0.168792,2.835777,0.036550,-0.308546,-0.473731,0.067973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MOAR012_CAL29_24H:P20,0.720000,0.089350,0.012950,-0.500700,0.337650,0.316625,0.299450,-0.705950,0.975199,-0.910450,...,-0.963500,-0.261600,1.784900,-0.120050,-0.014525,1.182849,0.200700,-0.805175,0.311750,-0.539150
MOAR012_CAL29_24H:P21,0.228000,0.163050,0.878400,0.325050,0.363550,-0.015475,-0.047050,-1.016950,0.911200,-0.180750,...,-0.806199,-0.280700,-0.057600,0.419049,0.086025,-0.730350,-0.054000,-0.219225,-0.068200,-0.620500
MOAR012_CAL29_24H:P22,1.011800,-0.041500,-0.527250,-0.282749,0.222350,0.847425,0.118650,0.719150,0.086600,-0.913651,...,-0.483850,-0.268950,0.182800,-0.475250,0.152575,-0.222650,0.165400,-0.000625,0.029600,-0.893850
MOAR012_CAL29_24H:P23,0.460600,-0.018550,-0.058700,-0.151549,0.265350,0.616323,0.253850,0.040050,0.322900,-0.266550,...,-0.682250,-0.032701,-0.262151,0.196850,0.215825,-0.714250,0.070750,-0.597925,-0.226200,-0.002400


In [14]:
Data_norm_df = Data_norm_df[Data_norm_df.index.isin(exp_ids)]

# arrange experiments in same order as in list
Data_norm_df["sort_col"] = Data_norm_df.index.map({val: i for i, val in enumerate(exp_ids)})
Data_norm_df = Data_norm_df.sort_values("sort_col")
Data_norm_df = Data_norm_df.drop("sort_col", axis = 1)

# transpose
Data_norm_df = Data_norm_df.T

display(Data_norm_df)

,ASG002_CAL29_24H:A10,ASG002_CAL29_24H:A11,ASG002_CAL29_24H:A12,ASG002_CAL29_24H:A19,ASG002_CAL29_24H:A20,ASG002_CAL29_24H:A21,ASG002_CAL29_24H:B03,ASG002_CAL29_24H:B10,ASG002_CAL29_24H:B11,ASG002_CAL29_24H:B12,...,MOAR011_CAL29_24H:C03,MOAR011_CAL29_24H:C10,MOAR011_CAL29_24H:C11,MOAR011_CAL29_24H:C12,MOAR011_CAL29_24H:F07,MOAR011_CAL29_24H:F08,MOAR011_CAL29_24H:F09,MOAR011_CAL29_24H:J10,MOAR011_CAL29_24H:J11,MOAR011_CAL29_24H:J12
AARS,-0.365007,0.137543,0.195393,-0.201708,-0.435808,0.104892,-0.292908,0.415293,0.090793,0.005292,...,-0.168651,0.092574,0.146474,-0.177126,0.612000,-0.108525,0.081124,0.066875,0.267425,0.110950
ABCB6,-0.787000,-0.881001,-0.515400,-0.109700,-0.513800,-0.246300,-0.189500,0.245300,-0.400250,-0.003700,...,-0.242962,0.460963,0.373238,0.299338,-0.070662,0.459338,-0.002812,0.295687,0.452038,0.646338
ABCC5,-0.884135,-0.747585,-0.280484,-0.559085,-0.343435,-1.042685,-0.942535,-0.586785,0.619615,0.534615,...,-0.797213,-0.357013,-0.677962,0.036688,-0.675037,0.169288,0.308537,-0.243187,-0.079063,-0.816362
ABCF1,0.759057,0.591608,-0.069793,0.418507,0.379807,0.099607,-0.348342,0.648608,-0.193393,-0.289292,...,-0.071975,0.306224,0.280474,0.489325,-0.300576,1.072275,-0.022275,-0.740876,-0.093725,-0.596925
ABCF3,0.143808,0.166508,0.133558,0.339407,0.333807,0.208508,-1.091042,0.252857,0.439207,0.291707,...,0.110974,0.310075,0.204076,0.108174,0.554524,0.386425,0.226425,-0.057725,0.071876,0.073976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF395,2.279827,1.028777,-0.232423,0.291577,2.945577,0.315477,0.227677,0.179877,-0.749623,-0.507123,...,-0.457799,-0.163800,0.925601,0.154900,-0.726600,-0.453500,-0.111000,-0.694400,0.258350,-0.589450
ZNF451,0.203850,-0.052300,0.206500,-0.036250,-0.197550,0.131350,-0.183850,0.081900,0.058851,0.029750,...,-0.012576,0.175375,-0.040325,0.186875,-0.096375,-0.088974,-0.094725,0.160525,0.039526,0.045975
ZNF586,-0.633796,-0.428896,4.301204,-0.355896,-0.128496,-0.206596,-0.107796,0.102704,-0.082096,-0.222996,...,0.198150,0.078900,-0.174200,-0.213800,-0.176550,-0.140200,-0.463200,-0.048450,-0.150750,0.068600
ZNF589,-0.080281,0.779069,-0.503131,-0.145930,-0.069031,-0.132631,0.209769,0.064369,0.138469,-0.292931,...,0.040037,0.038263,0.103963,0.219562,-0.200463,0.463113,0.177763,0.035163,0.117838,0.170112


In [15]:
genes = Data_norm_df.index.tolist()
print('Landmark genes list: ', len(genes), genes)

n_genes = len(genes)

Landmark genes list:  978 ['AARS', 'ABCB6', 'ABCC5', 'ABCF1', 'ABCF3', 'ABHD4', 'ABHD6', 'ABL1', 'ACAA1', 'ACAT2', 'ACBD3', 'ACD', 'ACLY', 'ACOT9', 'ADAM10', 'ADAT1', 'ADGRE5', 'ADGRG1', 'ADH5', 'ADI1', 'ADO', 'ADRB2', 'AGL', 'AKAP8', 'AKAP8L', 'AKR7A2', 'AKT1', 'ALAS1', 'ALDH7A1', 'ALDOA', 'ALDOC', 'AMDHD2', 'ANKRD10', 'ANO10', 'ANXA7', 'APBB2', 'APOE', 'APP', 'APPBP2', 'ARFIP2', 'ARHGAP1', 'ARHGEF12', 'ARHGEF2', 'ARID4B', 'ARID5B', 'ARL4C', 'ARNT2', 'ARPP19', 'ASAH1', 'ASCC3', 'ATF1', 'ATF5', 'ATF6', 'ATG3', 'ATMIN', 'ATP11B', 'ATP1B1', 'ATP2C1', 'ATP6V0B', 'ATP6V1D', 'AURKA', 'AURKB', 'AXIN1', 'B4GAT1', 'BACE2', 'BAD', 'BAG3', 'BAMBI', 'BAX', 'BCL2', 'BCL7B', 'BDH1', 'BECN1', 'BHLHE40', 'BID', 'BIRC2', 'BIRC5', 'BLCAP', 'BLMH', 'BLVRA', 'BMP4', 'BNIP3', 'BNIP3L', 'BPHL', 'BRCA1', 'BTK', 'BUB1B', 'BZW2', 'C2CD2', 'C2CD2L', 'C2CD5', 'C5', 'CAB39', 'CALM3', 'CALU', 'CAMSAP2', 'CANT1', 'CAPN1', 'CARMIL1', 'CASC3', 'CASK', 'CASP10', 'CASP2', 'CASP3', 'CASP7', 'CAST', 'CAT', 'CBLB', 'CBR1

## Inhibitor concentrations, IC50, and perturbation matrices

In [16]:
inhib_conc_matrix = np.zeros((n_modules, n_experiments))
ic50_matrix = np.ones((n_modules, n_experiments))
gamma_matrix = np.zeros((n_modules, n_experiments))

In [17]:

for i, module in enumerate(modules):
    drugs_for_module = IC50_df.Drug[IC50_df.Module == module].tolist()
    for drug in drugs_for_module:
        # get IC50 for this drug
        ic50 = IC50_df.IC50[IC50_df.Drug == drug].values
#       gamma = IC50_df.Gamma[IC50_df.Drug == drug].values
        print(drug, ic50)
        assert ic50.size == 1
#       assert gamma.size == 1
        # get experiments with this drug
        exp_with_drug = sig_info_df.index[sig_info_df.pert_drug == drug].tolist()
        print(exp_with_drug) 
        for exp_id in exp_with_drug:
            j = exp_ids.index(exp_id)
            print(j)
            # extract inhibitor concentration
            inhib_conc = sig_info_df.dose_float[sig_info_df.index == exp_id].values
            assert inhib_conc.size == 1
            # insert values in matrices
            inhib_conc_matrix[i, j] = inhib_conc.item()
            ic50_matrix[i, j] = ic50.item()
#           gamma_matrix[i, j] = gamma.item()


flufenamic-acid [3.]
['MOAR008_CAL29_24H:L01', 'MOAR008_CAL29_24H:L02', 'MOAR008_CAL29_24H:L03']
81
82
83
nandrolone [9.]
['MOAR011_CAL29_24H:J10', 'MOAR011_CAL29_24H:J11', 'MOAR011_CAL29_24H:J12']
113
114
115
oxandrolone [190.3]
['MOAR011_CAL29_24H:C01', 'MOAR011_CAL29_24H:C02', 'MOAR011_CAL29_24H:C03']
104
105
106
testosterone-enanthate [200.]
['MOAR011_CAL29_24H:C10', 'MOAR011_CAL29_24H:C11', 'MOAR011_CAL29_24H:C12']
107
108
109
testosterone-propionate [124.]
['MOAR010_CAL29_24H:L19', 'MOAR010_CAL29_24H:L20', 'MOAR010_CAL29_24H:L21']
101
102
103
JNJ-7706621 [0.027]
['ASG002_CAL29_24H:N13', 'ASG002_CAL29_24H:N14', 'ASG002_CAL29_24H:N15']
58
59
60
PHA-793887 [0.18]
['ASG002_CAL29_24H:L02', 'ASG002_CAL29_24H:L03']
47
48
roscovitine [2.]
['ASG002_CAL29_24H:E22', 'ASG002_CAL29_24H:E23', 'ASG002_CAL29_24H:E24']
21
22
23
alvocidib [0.12]
['ASG002_CAL29_24H:F05', 'ASG002_CAL29_24H:F06']
27
28
palbociclib [0.045]
['ASG002_CAL29_24H:P16', 'ASG002_CAL29_24H:P17', 'ASG002_CAL29_24H:P18']
76
77


In [18]:
# transform matrices into pandas dfs for export with row and column names
inhib_conc_df = pd.DataFrame(inhib_conc_matrix, index = modules, columns = exp_ids)
ic50_df = pd.DataFrame(ic50_matrix, index = modules, columns = exp_ids)
# gamma_df = pd.DataFrame(gamma_matrix, index = modules, columns = exp_ids)

# create binary perturbation matrix
pert_df = pd.DataFrame(
    np.where(inhib_conc_matrix != 0, 1, 0),
    index = inhib_conc_df.index,
    columns = inhib_conc_df.columns,
)

In [19]:
display(ic50_df)
# display(gamma_df)
display(inhib_conc_df)
display(pert_df)

,ASG002_CAL29_24H:A10,ASG002_CAL29_24H:A11,ASG002_CAL29_24H:A12,ASG002_CAL29_24H:A19,ASG002_CAL29_24H:A20,ASG002_CAL29_24H:A21,ASG002_CAL29_24H:B03,ASG002_CAL29_24H:B10,ASG002_CAL29_24H:B11,ASG002_CAL29_24H:B12,...,MOAR011_CAL29_24H:C03,MOAR011_CAL29_24H:C10,MOAR011_CAL29_24H:C11,MOAR011_CAL29_24H:C12,MOAR011_CAL29_24H:F07,MOAR011_CAL29_24H:F08,MOAR011_CAL29_24H:F09,MOAR011_CAL29_24H:J10,MOAR011_CAL29_24H:J11,MOAR011_CAL29_24H:J12
Androgen,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,190.3,200.0,200.0,200.0,1.00,1.00,1.00,9.0,9.0,9.0
CDK1,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.0,1.0,1.0
CDK2,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.0,1.0,1.0
CDK4_6,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.0,1.0,1.0
EGFR,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.0,1.0,1.0
Estrogen,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.0,1.0,1.0
FGFR,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.0,1.0,1.0
PI3K,0.00262,0.00262,0.00262,0.1595,0.1595,0.1595,0.2743,1.0000,1.0000,1.0000,...,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.0,1.0,1.0
p53,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,0.0018,0.0018,0.0018,...,1.0,1.0,1.0,1.0,62.15,62.15,62.15,1.0,1.0,1.0
TOP2A,1.00000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.0,1.0,1.0


,ASG002_CAL29_24H:A10,ASG002_CAL29_24H:A11,ASG002_CAL29_24H:A12,ASG002_CAL29_24H:A19,ASG002_CAL29_24H:A20,ASG002_CAL29_24H:A21,ASG002_CAL29_24H:B03,ASG002_CAL29_24H:B10,ASG002_CAL29_24H:B11,ASG002_CAL29_24H:B12,...,MOAR011_CAL29_24H:C03,MOAR011_CAL29_24H:C10,MOAR011_CAL29_24H:C11,MOAR011_CAL29_24H:C12,MOAR011_CAL29_24H:F07,MOAR011_CAL29_24H:F08,MOAR011_CAL29_24H:F09,MOAR011_CAL29_24H:J10,MOAR011_CAL29_24H:J11,MOAR011_CAL29_24H:J12
Androgen,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,1.11,10.0,3.33,1.11,0.0,0.00,0.00,10.0,3.33,1.11
CDK1,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00
CDK2,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00
CDK4_6,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00
EGFR,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00
Estrogen,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00
FGFR,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00
PI3K,10.0,1.11,0.12,10.0,1.11,0.12,0.12,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00
p53,0.0,0.00,0.00,0.0,0.00,0.00,0.00,10.0,1.11,0.12,...,0.00,0.0,0.00,0.00,10.0,3.33,1.11,0.0,0.00,0.00
TOP2A,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00


,ASG002_CAL29_24H:A10,ASG002_CAL29_24H:A11,ASG002_CAL29_24H:A12,ASG002_CAL29_24H:A19,ASG002_CAL29_24H:A20,ASG002_CAL29_24H:A21,ASG002_CAL29_24H:B03,ASG002_CAL29_24H:B10,ASG002_CAL29_24H:B11,ASG002_CAL29_24H:B12,...,MOAR011_CAL29_24H:C03,MOAR011_CAL29_24H:C10,MOAR011_CAL29_24H:C11,MOAR011_CAL29_24H:C12,MOAR011_CAL29_24H:F07,MOAR011_CAL29_24H:F08,MOAR011_CAL29_24H:F09,MOAR011_CAL29_24H:J10,MOAR011_CAL29_24H:J11,MOAR011_CAL29_24H:J12
Androgen,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,1,1,1
CDK1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDK2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDK4_6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
EGFR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Estrogen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FGFR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PI3K,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
p53,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,1,1,1,0,0,0
TOP2A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Global responses for DPD modules

According to our discussion, $R$ for DPD vectors can not be calculated with the same formula as for pathway activities. Instead we are assuming:

\begin{equation}
R_{DPD},j = DPD = STV_{DPD} \cdot Data_j
\end{equation}

In [22]:
# load STV data frame
STVs = pd.read_excel(os.path.join(info_dir, "ALL_DATA_2020_Jing.xlsx"), sheet_name = "STV", index_col = 0)
STV_df = pd.DataFrame(np.zeros((len(Data_norm_df.index), 3)), index = Data_norm_df.index, columns = STVs.columns)
STV_df.loc[STVs.index] = STVs

display(STV_df)

,blca_invasiveness,blca_oncogenesis,blca_survival
AARS,0.016052,0.078487,0.000000
ABCB6,0.001612,0.027125,-2.744978
ABCC5,-0.016742,0.002121,0.000000
ABCF1,-0.038990,-0.028959,-0.555168
ABCF3,0.023288,0.036671,0.000000
...,...,...,...
ZNF395,0.000000,0.000000,0.000000
ZNF451,0.011450,0.013403,0.000000
ZNF586,0.000000,0.000000,0.000000
ZNF589,0.000000,0.000000,0.000000


In [23]:
# create empty DPD data frame
DPD_df = pd.DataFrame(
    np.zeros((len(Data_norm_df.columns), len(STV_df.columns))),
    index = Data_norm_df.columns,
    columns = STV_df.columns,
)

# populate
for exp_id in DPD_df.index:
    for state in STV_df.columns:
        DPD_df.loc[exp_id, state] = np.dot(Data_norm_df.T.loc[exp_id], STV_df.loc[:, state])

display(DPD_df)

,blca_invasiveness,blca_oncogenesis,blca_survival
ASG002_CAL29_24H:A10,0.666350,-0.476596,-0.935466
ASG002_CAL29_24H:A11,0.176084,-0.582523,-7.009966
ASG002_CAL29_24H:A12,-0.202414,0.188991,10.710298
ASG002_CAL29_24H:A19,-0.689349,-1.251639,1.220367
ASG002_CAL29_24H:A20,-0.337604,-0.739846,8.697574
...,...,...,...
MOAR011_CAL29_24H:F08,-0.440833,0.443115,18.966249
MOAR011_CAL29_24H:F09,0.291640,0.754259,3.609794
MOAR011_CAL29_24H:J10,-0.731852,-0.320934,-4.148787
MOAR011_CAL29_24H:J11,0.208999,0.536319,-0.124785


In [24]:
# transform to R global
R_global_DPD_df = DPD_df.T
display(R_global_DPD_df)

,ASG002_CAL29_24H:A10,ASG002_CAL29_24H:A11,ASG002_CAL29_24H:A12,ASG002_CAL29_24H:A19,ASG002_CAL29_24H:A20,ASG002_CAL29_24H:A21,ASG002_CAL29_24H:B03,ASG002_CAL29_24H:B10,ASG002_CAL29_24H:B11,ASG002_CAL29_24H:B12,...,MOAR011_CAL29_24H:C03,MOAR011_CAL29_24H:C10,MOAR011_CAL29_24H:C11,MOAR011_CAL29_24H:C12,MOAR011_CAL29_24H:F07,MOAR011_CAL29_24H:F08,MOAR011_CAL29_24H:F09,MOAR011_CAL29_24H:J10,MOAR011_CAL29_24H:J11,MOAR011_CAL29_24H:J12
blca_invasiveness,0.666350,0.176084,-0.202414,-0.689349,-0.337604,0.147266,0.005792,-0.633114,-1.307159,0.022806,...,-0.081570,-0.331420,-0.667582,0.409147,0.191564,-0.440833,0.291640,-0.731852,0.208999,-0.528528
blca_oncogenesis,-0.476596,-0.582523,0.188991,-1.251639,-0.739846,-0.432395,0.644882,-0.456219,0.834357,0.100762,...,0.444948,-0.024804,0.428112,0.594009,1.343718,0.443115,0.754259,-0.320934,0.536319,0.130783
blca_survival,-0.935466,-7.009966,10.710298,1.220367,8.697574,-4.016413,-12.727442,-0.689528,3.421504,3.043562,...,-4.449759,8.672857,7.760388,-0.986666,6.163084,18.966249,3.609794,-4.148787,-0.124785,0.361812


## Save outputs

In [25]:
# save metadata as pickle
all_metadata = {
    "modules": modules,
    "n_modules": n_modules,
    "drugs": drugs,
    "n_drugs": n_drugs,
    "exp_ids": exp_ids,
    "n_experiments": n_experiments,
    "genes": genes,
    "n_genes": n_genes,
}

print(all_metadata)

with open(os.path.join(out_dir, "metadata.pickle"), "wb") as f:
    pickle.dump(all_metadata, f, protocol = pickle.HIGHEST_PROTOCOL)

{'modules': ['Androgen', 'CDK1', 'CDK2', 'CDK4_6', 'EGFR', 'Estrogen', 'FGFR', 'PI3K', 'p53', 'TOP2A'], 'n_modules': 10, 'drugs': ['serdemetan', 'testosterone-propionate', 'NVP-BEZ235', 'estradiol-cypionate', 'PI-103', 'LY-294002', 'roscovitine', 'mitoxantrone', 'alvocidib', 'RITA', 'erlotinib', 'masitinib', 'HLI-373', 'testosterone-enanthate', 'afatinib', 'GDC-0349', 'nandrolone', 'ponatinib', 'idarubicin', 'sorafenib', 'epirubicin', 'taselisib', 'daunorubicin', 'AS-605240', 'oxandrolone', 'dienestrol', 'vandetanib', 'raloxifene', 'gefitinib', 'flufenamic-acid', 'SAR405838', 'KU-0063794', 'lapatinib', 'PHA-793887', 'palbociclib', 'AZD-8055', 'nutlin-3', 'JNJ-7706621', 'AMG-232'], 'n_drugs': 39, 'exp_ids': ['ASG002_CAL29_24H:A10', 'ASG002_CAL29_24H:A11', 'ASG002_CAL29_24H:A12', 'ASG002_CAL29_24H:A19', 'ASG002_CAL29_24H:A20', 'ASG002_CAL29_24H:A21', 'ASG002_CAL29_24H:B03', 'ASG002_CAL29_24H:B10', 'ASG002_CAL29_24H:B11', 'ASG002_CAL29_24H:B12', 'ASG002_CAL29_24H:B13', 'ASG002_CAL29_24H:B

In [26]:
# save doses and perturbation matrix
inhib_conc_df.to_csv(os.path.join(out_dir, "inhib_conc_annotated.csv"))
ic50_df.to_csv(os.path.join(out_dir, "ic50_annotated.csv"))
# gamma_df.to_csv(os.path.join(out_dir, "gamma_annotated.csv"))
pert_df.to_csv(os.path.join(out_dir, "pert_annotated.csv"))

In [27]:
# save log fold change L1000 data
Data_norm_df.to_csv(os.path.join(out_dir, "L1000_Data_norm_data.csv"))

In [28]:
# save R_global for DPDs
R_global_DPD_df.to_csv(os.path.join(out_dir, "R_global_DPDonly_annotated.csv"))